In [1]:
import urllib.request as request
import json
import os
import pandas as pd
import datetime
import collections
import numpy as np

## Algorithm explanation
1. The main feature extracted from the keystroke data for every entry for every user is the time elapsed to type the given key since the last uncorrected key. So if backspace is used, then the time elapsed in typping the erase char, backspace and the new char are all included in the typing time of the new char. The reason for using this is that it is one of the simplest way of encoding corrections at different chars through the typing time which should have some user specific distribution.
2. The next feature used is just the raw count of backspaces typed by the user in every attempt.

In [2]:
def validateFeaturizeData(data, user_id, colnames):
    valid_str = 'Be Authentic. Be Yourself. Be Typing.'
    feature_length = len(valid_str) + 1 # 1 for number of backspaces.
    user_feature_df = pd.DataFrame(columns = colnames)
    invalid_count = 0
    invalid_idx = []
    total_count = len(data)
#     if total_count < 300:
#         print('insufficient data for user {}'.format(user_id))
#         return None, ()
    for idx, typed_str in enumerate(data):
        processed_str = []
        prev_time_stamp = None
        backstrokes = 0
        user_features = []
        for char_dict in typed_str:
            curr_char = char_dict["character"]
            if curr_char == "[backspace]":
                processed_str.pop() # remove the last char
                user_features.pop() # remove the typing time corresponding to last char
                backstrokes += 1
            else:
                curr_time_stamp = datetime.datetime.strptime(char_dict['typed_at'], '%Y-%m-%dT%H:%M:%S.%f')
                if prev_time_stamp:
                    typing_time = (curr_time_stamp - prev_time_stamp).total_seconds()
                else:
                    typing_time = 0.0
                processed_str.append(curr_char)
                prev_time_stamp = curr_time_stamp
                user_features.append(typing_time)       
            
        
        filtered_str = ''.join(processed_str)
        if filtered_str == valid_str:
            # append it to the dataframe.
            user_features.append(backstrokes)
            if len(user_features) != feature_length:
                print('Mismatch in feature length for a valid string.\n\
The original string is {}.\n\
The processed string is {}.\n\
The number of backspaces are {}.\n\
User features list is {}'.format(typed_str, processed_str, backstrokes, user_features))
            
            else:
                user_feature_df = user_feature_df.append(pd.Series(user_features, index = colnames), ignore_index = True)
        else:
#             print('invalid string: '+filtered_str)# do nothing and pass
            invalid_count += 1
            invalid_idx.append(idx)
    
    user_feature_df['user_id'] = user_id        
#     print(user_feature_df)
#     print(user_id, total_count, invalid_count)
    return user_feature_df, (total_count, invalid_count), invalid_idx

## Reading Train Data

Code to read the input data from the provided urls, filter valid strings and hence enroll eligible users and featurize their typing data and convert it to a dataframe

In [3]:
domain = 'https://challenges.unify.id/v1/mle'
total_users = 0
next_user = 'user_4a438fdede4e11e9b986acde48001122.json'
url_error = 0
stats = collections.defaultdict(list)
unenrolled_users = []
no_features = len('Be Authentic. Be Yourself. Be Typing.')
col_names = ['f_{}'.format(i) for i in range(1, no_features+1)]
col_names += ['del', 'user_id']
train_data = pd.DataFrame(columns = col_names)
while next_user:
    url = os.path.join(domain, next_user)
    with request.urlopen(url) as response:
            if response.getcode() == 200:
                source = response.read()
                data = json.loads(source)
                total_users += 1
                user_df, user_stats, _ = validateFeaturizeData(data['user_data'], next_user, col_names[:-1])
                if user_stats[0] - user_stats[1] >= 300:
                    stats['user'].append(next_user)
                    stats['total_count'].append(user_stats[0])
                    stats['invalid_count'].append(user_stats[1])
                    train_data = train_data.append(user_df)
                else:
                    # user cannot be enrolled into production due to threshold.
                    unenrolled_users.append(next_user)
                
                next_user = data['next']
            else:
                print('An error occurred while attempting to retrieve data from the API. Invalid User hash {}'.format(next_user))
                url_error += 1
        

In [4]:
# List of enrolled and unenrolled Users
print('Enrolled users are \n{}'.format(stats['user']))
print('\nUnenrolled users are \n{}'.format(unenrolled_users))

Enrolled users are 
['user_4a438fdede4e11e9b986acde48001122.json', 'user_4a45245cde4e11e9b51bacde48001122.json', 'user_4a4558dcde4e11e9bbfaacde48001122.json', 'user_4a45bafade4e11e99e37acde48001122.json', 'user_4a45f03ade4e11e9922cacde48001122.json', 'user_4a4626c2de4e11e98c17acde48001122.json', 'user_4a466e5cde4e11e98e06acde48001122.json', 'user_4a46a7a8de4e11e99efeacde48001122.json', 'user_4a46debede4e11e996d6acde48001122.json', 'user_4a47153ade4e11e9a04eacde48001122.json', 'user_4a474bf6de4e11e980dbacde48001122.json', 'user_4a47885ade4e11e98c41acde48001122.json', 'user_4a482be8de4e11e9a2a2acde48001122.json', 'user_4a486266de4e11e98893acde48001122.json', 'user_4a4898e4de4e11e98573acde48001122.json', 'user_4a4905d8de4e11e99a50acde48001122.json', 'user_4a493c4ade4e11e99e33acde48001122.json', 'user_4a497764de4e11e9bc80acde48001122.json', 'user_4a49ae0ade4e11e9b2d3acde48001122.json', 'user_4a49e492de4e11e98206acde48001122.json', 'user_4a4a1b1ade4e11e9a868acde48001122.json', 'user_4a4a518

## Model architecture and Training
The model used here is a simple feed forward 3 layer neural network. The final layer of the model is a softmax that outputs the probability distribution over all the enrolled users.
While more complex models like LSTM are better suited for the task, this model forms an appropriate baseline since it is one of the most simple architecture with basic feature extraction and engineering. 

In [5]:
from keras.models import Sequential
from keras.layers import Dense
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
# from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.utils import shuffle
from keras import callbacks

In [6]:
no_classes = int(train_data.user_id.nunique())
train_data = shuffle(train_data)
X_train = train_data.iloc[:,1:-1]
Y_train = train_data.iloc[:,-1]
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y_train)
encoded_Y = encoder.transform(Y_train)
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping[-1] = 'invalid input'
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
 
# define baseline model
def baseline_model(input_dims, hidden_units, no_classes):
    # create model
    model = Sequential()
    model.add(Dense(hidden_units[0], input_dim=input_dims, activation='relu'))
    model.add(Dense(hidden_units[1], activation='relu'))
    model.add(Dense(no_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

model = baseline_model(no_features, [100, 50], no_classes)
print(model.summary())
checkpoint_filepath = os.path.join(os.getcwd(), 'model_checkpoint/')
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
model.fit(X_train, dummy_y, epochs=100, batch_size=1024, validation_split=0.1, callbacks=[model_checkpoint_callback])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               3800      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 35)                1785      
Total params: 10,635
Trainable params: 10,635
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
16/16 [==============================] - 0s 14ms/step - loss: 3.5481 - accuracy: 0.0590 - val_loss: 3.5253 - val_accuracy: 0.1098
Epoch 2/100
16/16 [==============================] - 0s 6ms/step - loss: 3.5231 - accuracy: 0.0803 - val_loss: 3.4997 - val_accuracy: 0.1127
Epoch 3/100
16/16 [==============================] - 0s 5ms/step - loss: 3.4976 - accuracy: 0.

16/16 [==============================] - 0s 5ms/step - loss: 1.4280 - accuracy: 0.4785 - val_loss: 1.4367 - val_accuracy: 0.5023
Epoch 54/100
16/16 [==============================] - 0s 4ms/step - loss: 1.4230 - accuracy: 0.4818 - val_loss: 1.4378 - val_accuracy: 0.4814
Epoch 55/100
16/16 [==============================] - 0s 5ms/step - loss: 1.4109 - accuracy: 0.4869 - val_loss: 1.4281 - val_accuracy: 0.5093
Epoch 56/100
16/16 [==============================] - 0s 4ms/step - loss: 1.4068 - accuracy: 0.4856 - val_loss: 1.4195 - val_accuracy: 0.4820
Epoch 57/100
16/16 [==============================] - 0s 4ms/step - loss: 1.3999 - accuracy: 0.4904 - val_loss: 1.4144 - val_accuracy: 0.4890
Epoch 58/100
16/16 [==============================] - 0s 4ms/step - loss: 1.3957 - accuracy: 0.4902 - val_loss: 1.4122 - val_accuracy: 0.5017
Epoch 59/100
16/16 [==============================] - 0s 5ms/step - loss: 1.3899 - accuracy: 0.4876 - val_loss: 1.4053 - val_accuracy: 0.5105
Epoch 60/100
16/16 

In [7]:
# User performance report
model.load_weights(checkpoint_filepath)
train_pred_class = np.argmax(model.predict(X_train), axis=-1)
perf_df = pd.DataFrame({'y_true' : encoded_Y, 'y_pred': train_pred_class})
perf_df['outcome'] = perf_df.y_true == perf_df.y_pred
perf_df = perf_df.groupby('y_true')['outcome'].agg(['sum', 'count'])
perf_df.reset_index(inplace = True)
perf_df['acc'] = perf_df['sum']/perf_df['count']
perf_df.to_csv(os.path.join(os.getcwd(),'user_level_perf.csv'), header = True, index=False)
perf_df.acc.describe()

count    35.000000
mean      0.498214
std       0.265494
min       0.012698
25%       0.309797
50%       0.544554
75%       0.708637
max       0.952978
Name: acc, dtype: float64

In [8]:
# Test Data
test_url = 'https://challenges.unify.id/v1/mle/sample_test.json'
with request.urlopen(test_url) as response:
    if response.getcode() == 200:
        source = response.read()
        data = json.loads(source)
        test_df, user_stats, invalid_idx = validateFeaturizeData(data['attempts'], 'test', col_names[:-1])

final_idx = []
for i, idx in enumerate(invalid_idx):
    final_idx.append(idx-i)

In [9]:
# predicting for the test data
X_test = test_df.iloc[:,1:-1]
# pred_class = model.predict_classes(X_test)
pred_class = np.argmax(model.predict(X_test), axis=-1)
# print(pred_class.shape)
pred_class = np.insert(pred_class, final_idx , -1)
pred_user = []
for user in pred_class:
    pred_user.append(mapping[user])

In [10]:
with open('final_pred.json', 'w') as fp:
    json.dump(pred_user, fp)
# json.dumps(pred_user)